In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


In [3]:
class NeuralNetwork(nn.Module): #需要继承nn.Module，实现__init__ + forward
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x): #向前传播定义
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([5], device='mps:0')


In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.3728, -0.1576, -0.0360, -0.1173,  0.5082, -0.2163, -0.4335,  0.6634,
          0.2087,  0.0189,  0.3991, -0.1295,  0.4029,  0.0283,  0.0984, -0.9538,
         -0.2615, -0.1723, -0.2503, -0.4817],
        [-0.2603, -0.0502, -0.0181,  0.2450,  0.5962, -0.2063, -0.5964,  0.5053,
         -0.1584, -0.1056, -0.0091,  0.1785,  0.2590,  0.2350,  0.0542, -1.0910,
          0.1545, -0.4436, -0.2667, -0.0841],
        [-0.3557,  0.2795,  0.0905,  0.0608,  0.2470, -0.1752, -0.2264,  0.0855,
          0.2543, -0.0881,  0.0362, -0.0377,  0.4889,  0.2563,  0.4651, -0.9437,
         -0.0849, -0.3524, -0.6745, -0.5228]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.5082, 0.0000, 0.0000, 0.6634, 0.2087,
         0.0189, 0.3991, 0.0000, 0.4029, 0.0283, 0.0984, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.2450, 0.5962, 0.0000, 0.0000, 0.5053, 0.0000,
         0.0000, 0.0000, 0.1785, 0.2590, 0.2350, 0.05

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0014,  0.0351,  0.0312,  ...,  0.0313,  0.0051, -0.0069],
        [ 0.0269, -0.0308,  0.0078,  ..., -0.0204,  0.0065,  0.0319]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0017,  0.0256], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0106,  0.0101, -0.0233,  ...,  0.0187,  0.0304, -0.0357],
        [-0.0130,  0.0078, -0.0346,  ...,  0.0382, -0.0042, -0.0088]],
       device='mps:0', grad_fn=<Slice